# Лабораторная работа №4
## Методы стохастической оптимизации
__Команда "Зелёные шапки"__ в составе: Заречнев Алексей, Петрасюк Алексей, Халили Алина, Галимова Ярослава (все - счастливые обитатели группы M3236)
***

В этой лабораторной работе от нас требовалось написать метод `иммитации отжига`. Этот метом получил своё название из-за схожести с физическим явлением, когда металл сначала сильно накаляют, а затем плавно снижают температуру, позволяя атомам занять оптимальные позиции. Таким образом сплав получается более прочным, так как теперь у кристаллической решётки минимум потенциальной энергии, и кристаллическая решётка становится более устойчивой. Во время охлаждения атом всё ещё может перейти в менее устойчивое положение, но с контроллируемым падением температуры эта вероятность уменьшается. Таким образом метод иммитации отжига находится на стыке разных наук: от физики и до теории вероятности. Этот метод очень хорош для зашумлённых или мультимоадльных функциях. Утверждается, что такой алгоритм позволяет нам с большой вероятностью избежать «застревания» в локальном минимуме. А почему это так, давайте разберёмся.

Благодаря этому методу была решена, например, задача Коммивояджера

Как и в предыдущих лабораторных работах, у нас была написанная нами библиотека `Optimization-lib`, лежащая в основе всех наших тестирований. В ней лежат все методы и классы для прошлых лабораторных работ, а также вспомогательные методы.

### Задание 1. Метод иммитации отжига

Данный метод очень поход на привычные нам методы отптимизации. Более того, он даже не требует никаких производных, то есть является методом первого порядка. Но в чём заключается его отличие?

У нас есть параметр `T` - текущая температура (проще говоря, это функция от номера итерации). На каждой итерации мы пытаемся сделать шаг в сторону некоторой точки neighbour.

+ Если ΔE = f(neighbor) - f(current) < 0, то всегда принимаем "соседа" (так как это даёт нам улучшение решения) 
+ Если ΔE > 0, то принимаем "худшее" решение с вероятностью: $P = e^{(-ΔE / T)}$

Температура же может меняться по какому-нибудь из законов. В нашем случае это будет линейная функция с гиперпараметром - тангенсом угла наклона, то есть скоростью убывания функции. Можно взять и другие (нелинейные) варианты.

In [1]:
import sys
import numpy as np
sys.path.insert(0, "./Optimization-lib")
from annealing import AnnealingDecent
from gradient_descent import GradientDecent
from graphics_plotter import GraphicsPlotter
from lrs import exponential_decay, gradient
from output import pretty_print, pretty_print_annealing
from function_wrapper import FunctionWrapper
from simulated_annealing import sa

func = FunctionWrapper(lambda x: x[0] ** 2 + 100 * x[1]**2)

gradient.clear()
func.clear()
gd = GradientDecent(exponential_decay(0.01, 0.001), func, [[-6.0, 6.0], [-6.0, 6.0]], 0.0001, gradient)
test_painter_1 = GraphicsPlotter(gd)
res_1 = gd.find_min([5, 3], 100000)
pretty_print(gd, "EXP GD", res_1, gradient)

an = AnnealingDecent(func, np.array([[-6.0, 6.0], [-6.0, 6.0]]), alpha=0.1, T0=1000)
test_painter_2 = GraphicsPlotter(an)
res_2 = an.find_min(10000)
pretty_print_annealing(an, "ANNEALING", res_2)


        EXP GD
            found result:         0.000053
            found result in:      ['0.00729093304433353108', '0.00000000000000000001']
            steps count:          392
            function calls count: 1569
            gradient calls count: 392
            hessian calls count:  0
          


c:\Users\yaros\OneDrive\Desktop\ITMO\Optimization-Methods\Optimization-Methods-lab4\Optimization-lib\annealing.py:52: RuntimeWarning: overflow encountered in scalar divide
  if delta < 0 or np.random.rand() < math.exp(-delta/T):
c:\Users\yaros\OneDrive\Desktop\ITMO\Optimization-Methods\Optimization-Methods-lab4\Optimization-lib\annealing.py:52: RuntimeWarning: divide by zero encountered in scalar divide
  if delta < 0 or np.random.rand() < math.exp(-delta/T):


[-0.019486948032190383, 0.0019029834722536169]


TypeError: unsupported format string passed to numpy.ndarray.__format__